In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_59206/3760276594.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [32]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test")\
       .config("spark.driver.extraClassPath", "/dados10t/datalake/raw/omop/docker_imagens/postgresql-42.6.0.jar")\
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

#### reading table

In [33]:
df = spark.read.parquet('/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_measurement_data.parquet')

In [21]:
df.printSchema()

root
 |-- visit_occurrence_id: integer (nullable = true)
 |-- measurement_date: date (nullable = true)
 |-- measurement_type_concept_id: integer (nullable = true)
 |-- measurement_concept_id: integer (nullable = true)
 |-- value_as_concept_id: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- measurement_id: integer (nullable = true)



In [28]:
df.select('measurement_concept_id', 'value_as_concept_id').show(10000)

+----------------------+-------------------+
|measurement_concept_id|value_as_concept_id|
+----------------------+-------------------+
|                757685|           45884084|
|                757685|           45878583|
|               3964944|           45884084|
|                757685|           45884084|
|                757685|           45878583|
|               3964944|           45884084|
|                757685|           45884084|
|               3964944|           45878583|
|               3964944|           45884084|
|               3964944|           45878583|
|               3964944|           45884084|
|               3964944|           45878583|
|               3964944|           45884084|
|                757685|           45878583|
|                757685|           45878583|
|                757685|           45878583|
|                757685|           45878583|
|                757685|           45878583|
|                757685|           45878583|
|         

In [34]:
options = {
    "url": "jdbc:postgresql://157.86.211.201:5432/postgres",
    "dbtable": "omop54_v2.measurement",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

In [9]:
#df = df.filter(F.col('condition_start_date').isNotNull())

In [35]:
df.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()